# Notes

Reason for using Selenium

The job search results page keeps adding more results at the bottom, but the URL stays the same. Therefore, it is not possible to loop through the pages. It is necessary to click on the next button until all the results are loaded and then scrape.

The url for a job search results page (for example for technology), when used later, redirects to the job search home page
without the search results. Therefore, it is necessary to type in the search term do a fresh search before scraping.

To do for this notebook

- see if just the qualifications part can be scraped.

Running the script 
    
- It takes about 10-15 minutes to run the part that scrapes the job titles and job description page links.
- It may take 3-4 hours to run the part that scrapes the job description text. It may take longer 
  if the script gets interruped for some reason. It is necessary for a person to check if the script run successfully 
  and rerun as needed.
- It looks like the script tends to get interrupted if the computer goes to sleep mode. 
  So it helps to keep the computer active for the script to complete running without interruption.

In [6]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By

# Get job titles and job description page links

In [7]:
# job_title = []
# job_link = []

#this url redirects to the page 'https://sjobs.brassring.com/TGnewUI/Search/Home/Home?partnerid=25633&siteid=5439&Codes=BeMore#home'
#url = 'https://sjobs.brassring.com/TGnewUI/Search/Home/Home?partnerid=25633&siteid=5439&Codes=BeMore#keyWordSearch=technology%20or%20software%20engineering%20or%20developer%20or%20azure%20or%20aws&locationSearch='

url = 'https://sjobs.brassring.com/TGnewUI/Search/Home/Home?partnerid=25633&siteid=5439&Codes=BeMore#home'

driver = webdriver.Chrome()

driver.implicitly_wait(20)

driver.get(url)
    
location = driver.find_element(By.XPATH,'//*[@id="initialSearchBox__26"]')
location.send_keys('usa')

search_button = driver.find_element(By.XPATH,'//*[@id="searchControls_BUTTON_2"]')
search_button.click()



In [8]:
x = 1

while x == 1:
    try:
    
        next_button = driver.find_element(By.XPATH,'//*[@id="showMoreJobs"]')
        next_button.click()

    except:
        x = 0    

In [9]:
job_title = []
job_link = []

In [10]:

y = 2
i = 1

while y != 0:  
    
    y = 2
    
    try:  
        job = driver.find_element(By.XPATH,'//*[@id="mainJobListContainer"]/div/div/ul/li['+str(i)+']/div[2]/div[1]')        
        job_title.append(job.text)
        
    except:
        job_title.append('')
        y -= 1

        
    try:
        link = driver.find_element(By.XPATH,'//*[@id="Job_'+str(i)+'"]')        
        job_link.append(link.get_attribute('href'))
        
    except:
        job_link.append('')
        y -= 1        

    i = i+1
    
    
print(len(job_title), len(job_link)) 
    


516 516


In [11]:
# convert to dataframe

df = pd.DataFrame(zip(job_title, job_link))
df.columns = ['TITLE', 'LINK']
df.head()

,TITLE,LINK
0,Spark Developer,https://sjobs.brassring.com/TGnewUI/Search/hom...
1,Team Lead - Service Desk,https://sjobs.brassring.com/TGnewUI/Search/hom...
2,Network Administrator-Vulnerability,https://sjobs.brassring.com/TGnewUI/Search/hom...
3,Java Production Support Analyst,https://sjobs.brassring.com/TGnewUI/Search/hom...
4,Service Desk Technician,https://sjobs.brassring.com/TGnewUI/Search/hom...


In [12]:
df['QUALIFICATIONS'] = np.nan
df.head()

,TITLE,LINK,QUALIFICATIONS
0,Spark Developer,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
1,Team Lead - Service Desk,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
2,Network Administrator-Vulnerability,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
3,Java Production Support Analyst,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
4,Service Desk Technician,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN


In [13]:
df.to_csv('infosys_jobs_usa_title_link.csv')

In [14]:
df = pd.read_csv('infosys_jobs_usa_title_link.csv')
df = df.drop(columns=['Unnamed: 0'], axis=1)
df.head()

,TITLE,LINK,QUALIFICATIONS
0,Spark Developer,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
1,Team Lead - Service Desk,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
2,Network Administrator-Vulnerability,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
3,Java Production Support Analyst,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN
4,Service Desk Technician,https://sjobs.brassring.com/TGnewUI/Search/hom...,NaN


# Get job description data

In [ ]:
# job description text - wording and how the text and the headings are organized differs among job postings
# the most commonality I could find is most of the job descriptions and qualifications are under li tags, 
# but not always. Therefore, I am grabbing all li tags from the job description pages.

In [ ]:
# get job description data in batches. It takes too long to get all at once.

In [15]:
driver = webdriver.Chrome()   
    
for i in range(len(df['LINK'])):
    
    try:
        job_text = ''
        url = (df['LINK'][i])

        driver.get(url)

        desc = driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[7]/div[4]/div[2]/div/div[3]/div[4]/p[2]')
        texts = desc.find_elements_by_tag_name('li')

        for Text in texts: 
            job_text = job_text+Text.text+' '

        df['QUALIFICATIONS'][i] = job_text

    except:
        df['QUALIFICATIONS'][i] = ''

driver.close()

C:\Users\meske\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:365: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\meske\AppData\Local\Temp/ipykernel_11552/1100658683.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['QUALIFICATIONS'][i] = job_text


In [16]:
df['QUALIFICATIONS'][500]

'Client relationship management and business development: lead client relationships, build a portfolio up to $10-30M, lead the opportunity management cycle: Prospect-Evaluate-Propose-Close Client delivery assurance: collaborate with all delivery partners involved to ensure fulfillment of all commitments to the client. Collaborate with the Delivery Manager to address all people or infrastructure related issues that may be affecting the delivery of the project vis-à-vis the specific client. Balance different projects running for the client that may involve different delivery managers or horizontal unit resources. Take go-to-market solutions to accounts and drive revenue from go-to-market solutions being sponsored by the business unit. Work closely with the Solutions Leader to build customized solutions pitches for the target account and driving the revenues and delivery of these solutions. Account planning and governance: build the account plan including relationships required, opportuni

In [17]:
len(df[df['QUALIFICATIONS'] == ''])


143

# Basic Cleaning

In [18]:
to_drop = df[df['QUALIFICATIONS'] == ''].index
df = df.drop(to_drop)
df = df.dropna()
len(df)


373

In [19]:
df['QUALIFICATIONS'] = df['QUALIFICATIONS'].str.lower()

In [20]:
df = df.drop_duplicates(subset=['TITLE', 'QUALIFICATIONS'])
len(df)

373

In [21]:
df = df.reset_index(drop=True)

In [22]:
df.head()

,TITLE,LINK,QUALIFICATIONS
0,Spark Developer,https://sjobs.brassring.com/TGnewUI/Search/hom...,knowledge of customer care operations experien...
1,Java Production Support Analyst,https://sjobs.brassring.com/TGnewUI/Search/hom...,"active listening and questioning, an eye for d..."
2,Technology Lead- React JS,https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor's degree or foreign equivalent requir...
3,Privacy Analyst,https://sjobs.brassring.com/TGnewUI/Search/hom...,candidate can be located anywhere in usa. this...
4,Network Administrator,https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor’s degree or foreign equivalent requi...


In [23]:
df['COMPANY'] = 'Infosys'
df['DESCRIPTION'] = df['QUALIFICATIONS']
df.head()

,TITLE,LINK,QUALIFICATIONS,COMPANY,DESCRIPTION
0,Spark Developer,https://sjobs.brassring.com/TGnewUI/Search/hom...,knowledge of customer care operations experien...,Infosys,knowledge of customer care operations experien...
1,Java Production Support Analyst,https://sjobs.brassring.com/TGnewUI/Search/hom...,"active listening and questioning, an eye for d...",Infosys,"active listening and questioning, an eye for d..."
2,Technology Lead- React JS,https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor's degree or foreign equivalent requir...,Infosys,bachelor's degree or foreign equivalent requir...
3,Privacy Analyst,https://sjobs.brassring.com/TGnewUI/Search/hom...,candidate can be located anywhere in usa. this...,Infosys,candidate can be located anywhere in usa. this...
4,Network Administrator,https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor’s degree or foreign equivalent requi...,Infosys,bachelor’s degree or foreign equivalent requi...


In [24]:
df.to_csv('infosys_usa_ jobs.csv')

In [25]:
df = pd.read_csv('infosys_usa_ jobs.csv')
df = df.reindex(columns=['COMPANY', 'TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION'])
df.head()

,COMPANY,TITLE,QUALIFICATIONS,LINK,DESCRIPTION
0,Infosys,Spark Developer,knowledge of customer care operations experien...,https://sjobs.brassring.com/TGnewUI/Search/hom...,knowledge of customer care operations experien...
1,Infosys,Java Production Support Analyst,"active listening and questioning, an eye for d...",https://sjobs.brassring.com/TGnewUI/Search/hom...,"active listening and questioning, an eye for d..."
2,Infosys,Technology Lead- React JS,bachelor's degree or foreign equivalent requir...,https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor's degree or foreign equivalent requir...
3,Infosys,Privacy Analyst,candidate can be located anywhere in usa. this...,https://sjobs.brassring.com/TGnewUI/Search/hom...,candidate can be located anywhere in usa. this...
4,Infosys,Network Administrator,bachelor’s degree or foreign equivalent requi...,https://sjobs.brassring.com/TGnewUI/Search/hom...,bachelor’s degree or foreign equivalent requi...


In [26]:
df.to_csv('infosys_usa_ jobs.csv')